In [1]:
from dotenv import load_dotenv
import os

# .env 불러오기
load_dotenv()

# 기본 모델 설정
# 여기서 설정할 수도 있지만 .env 내에 넣어줘도 됨
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai import Crew, Task

In [3]:
from crewai.tools import tool

import yfinance as yf
import requests


class Tools:

    @tool("Get ticker from company name")
    def get_ticker_from_name(company_name: str) -> str:
        """
        Get the stock ticker from the company name.
        This function will return the ticker for a given company name.
        """
        return get_ticker(company_name)

    # 일반 함수가 아닌 툴로써 작동하기 위해서는 @tool("툴의 역할")을 반드시 기입
    @tool("One month stock price history")
    def stock_price(ticker: str) -> str:
        # 무슨 일을 하는지 input으로 무엇이 주어져야 하는지 설명하는 것이 매우 중요함
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example AAPL, NET, TSLA etc...
        """
        # yFinance API 이용
        stock = yf.Ticker(ticker)
        # .to_csv(): csv 형식으로 반환함
        return stock.history(period="1mo").to_csv()

    @tool("Stock news URLs")
    def stock_news(ticker: str) -> str:
        """
        Useful to get URLs of news articles related to a stock.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        # lambda는 입력된 값을 그대로 내보냄. map을 사용해 stock.news["link"]를 찾아 link list를 만듦.
        return list(map(lambda x: x["link"], stock.news))

    @tool("Company's income statement")
    def income_stmt(ticker: str) -> str:
        """
        Useful to get the income statement of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @tool("Balance sheet")
    def balance_sheet(ticker: str) -> str:
        """
        Useful to get the balance sheet of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @tool("Get insider transactions")
    def insider_transactions(ticker: str) -> str:
        """
        Useful to get insider transactions of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()


def get_ticker(company_name: str) -> str:
    """
    회사 이름을 입력하면 Yahoo Finance에서 티커(symbol)를 찾아 반환.
    """
    url = f"https://query2.finance.yahoo.com/v1/finance/search?q={company_name}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        ticker = data["quotes"][0]["symbol"]  # 첫 번째 검색 결과의 티커 반환
        return ticker
    except:
        return None  # 검색 결과 없음

In [4]:
from crewai import Agent


from crewai_tools import ScrapeWebsiteTool


class Agents:

    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            # 주가를 가져오는 툴
            tools=[
                Tools.get_ticker_from_name,
                Tools.stock_price,
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory="You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            # 주식 뉴스, 링크 스크래핑
            tools=[
                Tools.get_ticker_from_name,
                Tools.stock_news,
                ScrapeWebsiteTool(),
            ],
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory="You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            # 대차대조표, 재무제표, 내부자거래
            tools=[
                Tools.get_ticker_from_name,
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory="You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
        )

In [5]:
from crewai import Task
from datetime import datetime


class Tasks:

    def research(self, agent, ticker):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment.",
            expected_output=f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance. Make sure to write the report in markdown format.",
            agent=agent,
            output_file=self.get_output_file(
                ticker,
                "stock_news.md",
            ),
        )

    def technical_analysis(self, agent, ticker):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions. Make sure to write the report in markdown format.",
            agent=agent,
            output_file=self.get_output_file(
                ticker,
                "technical_analysis.md",
            ),
        )

    def finacial_analysis(self, agent, ticker):
        return Task(
            description="Analyze {company}'s financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            expected_output=f"Your final answer MUST be a detailed financial analysis report that includes insights on the company's financial health, performance, and valuation. Provide an overview of the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential. Make sure to write the report in markdown format.",
            agent=agent,
            output_file=self.get_output_file(
                ticker,
                "financial_analysis.md",
            ),
        )

    def investment_recommendation(self, agent, context, ticker):
        return Task(
            description="Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.",
            expected_output=f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports. Make sure to write the report in markdown format.",
            agent=agent,
            context=context,
            output_file=self.get_output_file(
                ticker,
                "investment_recommendation.md",
            ),
        )

    def get_output_file(self, ticker, file_name):
        today = datetime.today().strftime("%m%d%Y")
        output_folder = f"{ticker}/{today}"
        # date_folder = f"{today}"
        os.makedirs(output_folder, exist_ok=True)
        # os.makedirs(date_folder, exist_ok=True)
        return f"{output_folder}/{file_name}"

In [6]:
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

company_name = "rezolve ai"
ticker = get_ticker(company_name)
if ticker is None:
    ticker = company_name

agents = Agents()
tasks = Tasks()

# class로 만들어 놓은 agent 할당
researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

# task에 agent를 할당
research_task = tasks.research(
    researcher,
    ticker,
)
technical_task = tasks.technical_analysis(
    technical_analyst,
    ticker,
)
financial_task = tasks.finacial_analysis(
    financial_analyst,
    ticker,
)
recommend_task = tasks.investment_recommendation(
    # agent
    hedge_fund_manager,
    # context: 리스트에 있는 모든 태스크가 완료되어야 이 태스크를 실행
    [
        technical_task,
        research_task,
        financial_task,
    ],
    ticker,
)

crew = Crew(
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager,
    ],
    tasks=[
        research_task,
        technical_task,
        financial_task,
        recommend_task,
    ],
    verbose=True,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=True,
)

result = crew.kickoff(
    inputs=dict(
        company=company_name,
    ),
)

# Agent: Crew Manager
## Task: Gather and analyze the latest news and market sentiment surrounding the stock of rezolve ai. Provide a summary of the news and any notable shifts in market sentiment.


# Agent: Crew Manager
## Using tool: Get ticker from company name
## Tool Input: 
"{\"company_name\": \"Rezolve AI\"}"
## Tool Output: 
RZLV


# Agent: Crew Manager
## Using tool: Stock news URLs
## Tool Input: 
"{\"ticker\": \"RZLV\"}"
## Tool Output: 
['https://finance.yahoo.com/news/rezolve-ai-limited-rzlv-gains-195039979.html', 'https://finance.yahoo.com/news/rezolve-ai-kicks-off-2025-130000607.html', 'https://finance.yahoo.com/news/rezolve-ai-gets-30-million-144518018.html', 'https://finance.yahoo.com/news/rezolve-ai-secures-30-million-130000019.html', 'https://finance.yahoo.com/news/rezolve-ai-limited-rzlv-positions-140330625.html', 'https://finance.yahoo.com/news/rezolve-ai-defies-ai-cost-130000422.html', 'https://finance.yahoo.com/news/rezolve-ai-looking-capitalize-apos-141618727.h